# BEVFormer 模型结构理解
## 一、模型核心目标与解决的问题
BEVFormer 是针对自动驾驶场景中多摄像头 3D 视觉感知任务设计的框架，核心目标是通过统一的鸟瞰图（BEV）特征表示，实现 3D 检测和地图分割等任务。其解决的关键问题包括：
多摄像头信息融合：传统单目框架单独处理不同视角图像，无法捕捉跨摄像头关联，导致性能受限；
高质量 BEV 特征生成：从 2D 图像生成 BEV 特征依赖深度信息，而深度准确性对 3D 检测至关重要，传统方法易受深度误差影响；
时间信息利用：现有方法很少融合历史帧信息，难以估计移动对象速度或检测遮挡目标。
## 二、整体框架概述
BEVFormer 的整体框架分为输入层、特征提取层、Transformer 解码层三部分，最终生成统一的 BEV 特征以支持多任务。
1. 输入层
输入数据格式：6 维张量(bs, queue, cam, C, H, W)，其中：
bs：批次大小；
queue：连续帧数量（含历史帧，用于时序建模）；
cam：每帧图像数量（如 nuScenes 数据集为 6）；
C, H, W：图像通道数、高度、宽度。
2. 特征提取层
骨干网络（Backbone）：使用 ResNet、Swin 等网络提取多尺度图像特征（如 ResNet 的C2, C3, C4, C5层）；
特征金字塔网络（FPN）：对骨干网络输出的特征进行多尺度融合，生成P2, P3, P4, P5特征图，为后续 Transformer 提供丰富的空间和语义信息。
FPN 输出层	尺寸 (H×W)	通道数	说明
P2	200×352	256	低层特征，空间分辨率高，保留细节
P3	100×176	256	中层特征，平衡细节与语义
P4	50×88	256	高层特征，语义信息强
P5	25×44	256	最高层特征，语义最强
3. Transformer 解码层
Transformer 解码层是 BEV 特征生成的核心，由 6 层重复模块组成，每层包含 3 个关键组件：
（1）前馈神经网络（FFN）
基础特征处理模块，负责特征变换与线性映射，通过两层全连接层增强特征表达能力，含 dropout（0.1）防止过拟合。
（2）空间交叉注意力机制（Spatial Cross-Attention）
核心思想：建立 BEV 空间与图像空间的关联，将 BEV 中的 query 点映射到多视角图像中，聚合对应像素特征；
关键步骤：
将 BEV 中的点(x', y')投影到各视角图像平面（cross-view projection）；
筛选出包含该点的有效视角V_hit；
通过多视角注意力机制，汇聚有效视角的图像特征到 BEV query 点。
（3）时间自注意力机制（Temporal Self-Attention）
核心思想：利用历史帧 BEV 特征B_{t-1}增强当前帧 BEV 特征B_t，建模时序动态性；
功能：
实现历史信息向当前帧的迁移，保证目标在连续帧中的一致性；
提升对遮挡物、低纹理目标的感知能力。
## 三、Encoder 与 Decoder 结构细节
1. Encoder（BEVFormerEncoder）
输入：多视角图像特征（FPN 输出）、历史 BEV 特征B_{t-1}；
核心功能：将多视角空间特征与历史时序特征融合，生成当前帧 BEV 特征B_t；
关键组件：
Temporal Self-Attention：处理历史 BEV 特征与当前 query 的关联，捕捉时序依赖；
Spatial Cross-Attention：融合多视角图像特征到 BEV 空间；
参考点生成：生成 3D 参考点（用于空间注意力）和 2D 参考点（用于时间注意力）。
2. Decoder（DetectionTransformerDecoder）
输入：Encoder 生成的 BEV 特征B_t、目标查询（Object Queries）；
核心功能：从 BEV 特征中提取目标信息，预测 3D 边界框和类别；
关键步骤：
初始化参考点（目标位置的初始猜测）；
迭代精化：通过自注意力（query 间交互）和交叉注意力（query 与 BEV 特征交互）更新目标查询；
输出：每层解码器的中间结果，最终生成 3D 边界框参数（cx, cy, cz, w, h, d, θ, vx, vy）和类别概率。
## 四、损失函数设计
BEVFormer 的损失函数用于优化 3D 目标检测任务：
分类损失：采用 Focal Loss，解决类别不平衡问题，公式为：
L 
focal
​
 =−α 
t
​
 (1−p 
t
​
 ) 
γ
 log(p 
t
​
 )

其中p_t为预测概率，α_t平衡正负样本，γ调节难易样本权重；
回归损失：采用 L1 Loss，衡量 3D 边界框参数（位置、尺寸、角度、速度）的预测误差；
集合匹配：使用匈牙利算法进行预测框与真实框的二分图匹配。
## 五、核心创新点
时空注意力融合：通过空间交叉注意力融合多摄像头空间信息，时间自注意力融合历史时序信息，实现 “空间 + 时间” 双维度建模；
统一 BEV 特征：生成的 BEV 特征可同时支持 3D 检测、地图分割等多任务，减少多模型训练成本；
高效特征映射：基于可变形注意力机制（MSDeformableAttention3D），仅关注关键区域，降低计算复杂度。
## 六、总结
BEVFormer 通过 Transformer 架构将多视角图像特征与历史时序特征融合为统一的 BEV 特征，解决了自动驾驶中 3D 感知的空间对齐和时序建模难题。其在 nuScenes 和 Waymo 等数据集上的实验表明，在 NDS 指标上达到新的最佳水平，尤其在速度估计和低可见度目标检测上表现出色。